In [2]:
from ete3 import Tree
from ete3 import PhyloTree
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt

import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures


from sklearn.cluster import KMeans, DBSCAN

# Just Simple Correlation

In [22]:
data = {
    'LGL-1': 'required_tree_OG0006648_species.txt',
    'PAR-1': 'required_tree_OG0000002_species.txt',
    'PKC-3': 'required_tree_OG0000300_species.txt',
    'PAR-2': 'required_tree_OG0000571_tree_species.txt'
}

# #For PAR-1
# species_1_list = ['PAR-1']
# species_2_list = ['PKC-3', 'LGL-1','PAR-2']

# #For PAR-2
# species_1_list = ['PAR-2']
# species_2_list = ['LGL-1']

#For PKC-3
species_1_list = ['PKC-3']
species_2_list = ['LGL-1','PAR-2']

for SPECIES_1 in species_1_list:
    for SPECIES_2 in species_2_list:

        species1_tree = PhyloTree(data[SPECIES_1], format=1)
        species2_tree = PhyloTree(data[SPECIES_2], format=1)

        species1 = species1_tree.cophenetic_matrix()
        species2 = species2_tree.cophenetic_matrix()

        species1_df = pd.DataFrame(species1[0], columns = species1[1])
        species1_df.index = species1[1]
        species2_df = pd.DataFrame(species2[0], columns = species2[1])
        species2_df.index = species2[1]
        common = np.intersect1d(species1_df.columns, species2_df.columns)
        species1_df = species1_df[species1_df.index.isin(common)]
        species1_df= species1_df[common]
        species2_df = species2_df[species2_df.index.isin(common)]
        species2_df= species2_df[common]


        r = (species1_df.dot(species2_df).sum().sum())/(np.sqrt(((species1_df-species1_df.mean().mean())**2).sum().sum())*np.sqrt(((species2_df-species2_df.mean().mean())**2).sum().sum()))

        x = species1_df.stack().reset_index()
        x["labels"] = x["level_0"] +' '+ x["level_1"]
        x['{}_values'.format(SPECIES_1)] = x[0]

        y = species2_df.stack().reset_index()
        y["labels"] = y["level_0"] +' '+ y["level_1"]
        y['{}_values'.format(SPECIES_2)] = y[0]

        xy = pd.merge(x, y, how="inner",on=['labels', 'level_0', 'level_1'] ).round(2)

        ##Ridge regression
        for species in xy['level_0'].unique():
            species_df = xy[xy['level_0']==species]
            X =  species_df['{}_values'.format(SPECIES_1)].values.reshape(-1,1)
            Y = species_df['{}_values'.format(SPECIES_2)].values.reshape(-1,1)

            model = Ridge(fit_intercept=True)
            model.fit(X,Y)
            xy.loc[xy['level_0']==species, 'R2'] = model.score(X, Y)


        ##Cluster based on R2 values
        data_cluster = xy['R2']

        clustering = KMeans(n_clusters=3, random_state=0).fit(data_cluster.values.reshape(-1,1))

        xy['clusters'] = clustering.labels_.astype(str)

        for i in clustering.labels_.astype(str):
            xy.loc[xy['clusters']==i,'clusters_R2'] = str(xy.loc[xy['clusters']==i,'R2'].round(2).min())+' to ' + str(xy.loc[xy['clusters']==i,'R2'].round(2).max())
            xy.loc[xy['clusters']==i,'std_clusters_{}'.format(SPECIES_1)] = xy.loc[xy['clusters']==i,'{}_values'.format(SPECIES_1)].round(2).std()
            xy.loc[xy['clusters']==i,'std_clusters_{}'.format(SPECIES_2)] = xy.loc[xy['clusters']==i,'{}_values'.format(SPECIES_2)].round(2).std()
            xy.loc[xy['clusters']==i,'mean_clusters_{}'.format(SPECIES_1)] = xy.loc[xy['clusters']==i,'{}_values'.format(SPECIES_1)].round(2).mean()
            xy.loc[xy['clusters']==i,'mean_clusters_{}'.format(SPECIES_2)] = xy.loc[xy['clusters']==i,'{}_values'.format(SPECIES_2)].round(2).mean()




#         fig = px.scatter(xy, x="{}_values".format(SPECIES_1), y="{}_values".format(SPECIES_2), color = 'clusters_R2',
#                          hover_name="labels", hover_data=['{}_values'.format(SPECIES_1), '{}_values'.format(SPECIES_2), 'R2'],
#                         )

#         print(xy.pivot_table('R2',['level_0'],'clusters_R2').fillna(''))
#         xy.pivot_table('R2',['level_0'],'clusters_R2').fillna('').round(2).to_csv("{}_{}_plot.csv".format(SPECIES_1,SPECIES_2))
#         fig.show()
#         fig.write_html("{}_{}_plot.html".format(SPECIES_1,SPECIES_2))



        fig = go.Figure()
        fig.add_trace(go.Bar(
            name='{}'.format(SPECIES_1),
            x=xy["clusters_R2"].unique().tolist(), y=xy["mean_clusters_{}".format(SPECIES_1)].round(2).unique().tolist(),
            error_y=dict(type='data', array=xy['std_clusters_{}'.format(SPECIES_1)].round(2).unique().tolist())  
        ))
        fig.add_trace(go.Bar(
            name='{}'.format(SPECIES_2),
            x=xy["clusters_R2"].unique().tolist(), y=xy["mean_clusters_{}".format(SPECIES_2)].round(3).unique().tolist(),
            error_y=dict(type='data', array=xy['std_clusters_{}'.format(SPECIES_2)].round(3).unique().tolist())
        ))
        fig.update_layout(barmode='group')
        fig.update_layout(
            title="Distribution of Distances in R2 clusters",
            xaxis_title="R2 clusters",
            yaxis_title="Distances",
            legend_title="Species",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="RebeccaPurple"
            )
        )

        fig.show()
        fig.write_html("{}_{}_cluster_distance_distribution.html".format(SPECIES_1,SPECIES_2))

        
        
        